In [1]:
import pandas as pd
import os
import glob
import re

In [2]:
## Declare constants

election_year = 2018

In [3]:
## Get all file paths of scraped raw CSVs

folders_list = glob.glob(os.path.join(os.path.dirname("__file__"),'data/raw/the_star/*'))

In [4]:
## Collect all the similar types of CSV files into seperate lists

state_result_paths = [path for path in folders_list if "state_result" in path]
parliament_result_paths = [path for path in folders_list if "parliament_result" in path]
state_fact_paths = [path for path in folders_list if "state_fact" in path]
parliament_fact_paths = [path for path in folders_list if "parliament_fact" in path]

In [5]:
def append_all(paths):
    '''
    Append CSVs into a dataframe.
    '''
    
    li = []

    for filename in paths:
        df = pd.read_csv(filename, index_col=None, header=0)
        li.append(df)

    df = pd.concat(li, axis=0, ignore_index=True)
    return df

In [6]:
def make_3_digit(num):
    '''
    Make int into 3 digit string with 0s as prefix
    '''
    
    digits = str(num)
    if num < 10:
        digits = '00' + str(num)
    elif num < 100:
        digits = '0' + str(num)
        
    return digits

In [7]:
def parse_comma_sep_num(num_string):
    '''
    Make number string with comma seperators into number
    '''

    if '.' in str(num_string):
        return float(''.join(str(num_string).split(',')))
    else:
        return int(''.join(str(num_string).split(',')))

In [8]:
def fix_typos(typos_list,data_row):
    '''
    Make replacements for typos in typos_list for each row
    typos_list is a list of dicts {key,old_value,new_value}
    '''
    new_row = data_row.copy()
    
    for typo in typos_list:
        new_row[typo['key']] = new_row[typo['key']].replace(typo['old_value'],typo['new_value'])
        
    return new_row

## Clean Votes Data

In [9]:
def clean_consituency_code(row):
    '''
    Clean the append Parliament seat code to State seat code to XXX or XXX-XXX respectively
    '''
    
    ## Clean parliament/state code
    ## e.g. P10, P201

    parliament_code_int = int(row['panel_code'].replace('P','').replace('N',''))
    parliament_code_digits = make_3_digit(parliament_code_int)

    ## For state constituencies, need to append Parliament seat code to State seat code PXXX-NXXX

    if 'N' in row['panel_code']:
        p_code_regex = re.compile(r"\(P(?P<p_code>[0-9]+)\)")
        p_code_info = p_code_regex.search(row['panel_name'])
        p_code_groups = p_code_info.groupdict()
        #print(row['panel_name'])
        parliament_p_code = p_code_groups['p_code']
        parliament_p_code_int = int(parliament_p_code)
        parliament_p_code_digits = make_3_digit(parliament_p_code_int)
        parliament_code_digits = parliament_p_code_digits + '-' + parliament_code_digits
        
    return parliament_code_digits

In [10]:
def clean_votes(raw_df):
    '''
    Clean votes data
    '''
    
    clean_df = pd.DataFrame(columns=['name','coalition','party_code','votes','vote_share',
                                    'parliament_code_digits','constituency', 'state'])
    
    for index, row in raw_df.iterrows():

        ## Clean name column 
        ## e.g. Mordi Bimol (PH - DAP) 
        ## e.g. Hamdan Sani (PAS)
        name_regex = re.compile(
            r"(?P<candidate>[a-zA-Z @]+\S)\s*(?:\(\s*(?P<coalition>[a-zA-Z' ]+\S)\s*-\s*(?P<party>[a-zA-Z' ]+)\s*\)|\(\s*(?P<party_only>[a-zA-Z' ]+\S)\s*\))"
        )
        name_info = name_regex.search(row['name'])
        name_groups = name_info.groupdict()

        name = name_groups['candidate']
        coalition = name_groups['coalition'] if name_groups['coalition'] != None else ''
        party_code = name_groups['party'] if name_groups['party'] != None else name_groups['party_only']
        
        ## Clean votes column
        ## e.g. 43.1% (12,771)
        
        vote_regex = re.compile(r"(?P<vote_share>[\d.]+)%\s\((?P<votes>[\d,]+)\)")
        vote_info = vote_regex.search(row['number_of_voters'])
        vote_groups = vote_info.groupdict()
        
        ## Star's vote share calculation shows share as a percentage of registered voters 
        ## instead of total vote. So we will calculate it ourselves below.
        vote_share_of_registered_voters = vote_groups['vote_share']
        
        votes = parse_comma_sep_num(vote_groups['votes'])
        
        
        ## Clean parliament/state code
        ## e.g. P10, P201
        parliament_code_digits = clean_consituency_code(row)
        
        ## Clean consituency name
        constituency = row['panel_name']
        
        ## Clean state
        ## Split upper camel case string into individual words
        state = ''.join(' ' + char if char.isupper() else char.strip() for char in row['state_region']).strip()
        
        
        new_row = {'name':name, 'coalition':coalition, 'party_code':party_code, 
                  'votes':votes, 'vote_share_of_registered_voters':vote_share_of_registered_voters,
                  'parliament_code_digits':parliament_code_digits,
                  'constituency':constituency, 'state':state}
        
                
        ## Clean Typos: 
        ### "state" column: "Negri Sembilan" -> "Negeri Sembilan"
        
        new_row = fix_typos([
            { 'key': 'state', 'old_value': 'Negri Sembilan', 'new_value': 'Negeri Sembilan'}
        ],new_row)
        
        clean_df = clean_df.append(new_row, ignore_index=True)
        

    clean_df['votes'] = pd.to_numeric(clean_df['votes'])
    
    
    ## Get winner and total votes for each constituency
    
    total_votes_list = clean_df.groupby(["parliament_code_digits"])["votes"].sum().reset_index()
    total_votes_list.rename(columns={'votes':'total_votes'},inplace=True)
    
    winner_list = clean_df.loc[clean_df.groupby('parliament_code_digits')['votes']
                               .agg(lambda x: x.idxmax())][['parliament_code_digits','party_code']]
    winner_list.rename(columns={'party_code':'winner'},inplace=True)
    
    clean_df_merged_with_winner = pd.merge(clean_df, winner_list, how="left", on=["parliament_code_digits"])
    clean_df_merged_with_winner['winner'] = clean_df_merged_with_winner \
        .apply(lambda row : 1 if row['party_code'] == row['winner'] else 0, axis = 1)
    
    clean_df_merged_with_total = pd.merge(clean_df_merged_with_winner, total_votes_list, 
                                          how="left", on=["parliament_code_digits"])
    
    ## Star's vote share calculation shows share as a percentage of registered voters 
    ## instead of total vote. So we will calculate it ourselves
    clean_df_merged_with_total['vote_share'] = clean_df_merged_with_total \
         .apply(lambda row: 100*row['votes']/row['total_votes'] if row['total_votes'] != 0 else 0,
                axis = 1) 

        
        
    return clean_df_merged_with_total

In [11]:
state_votes = clean_votes(append_all(state_result_paths))

In [12]:
parliament_votes = clean_votes(append_all(parliament_result_paths))

In [13]:
state_votes.to_csv('data/cleaned/the_star/results_'+str(election_year)+'_states.csv')

In [14]:
parliament_votes.to_csv('data/cleaned/the_star/results_'+str(election_year)+'_parliament.csv')

## Clean Constituency Facts Data

In [15]:
def clean_demographics(demo_string):
    '''
    Parse ethnicity percentage string and return dict of percent per ethnicity
    '''
    
    individual_pairs = demo_string.split("; ")
    demographics_dict = {}
    for pair in individual_pairs:
        demographics_regex = re.compile(r"(?P<ethnicity>[a-zA-Z /.()]+)\s(?P<percentage>[\d.]+)")
        demographics_info = demographics_regex.search(pair)
        demographics_groups = demographics_info.groupdict()
        ethnicity = demographics_groups['ethnicity']
        percentage = demographics_groups['percentage']
        demographics_dict['ethnicity_'+ethnicity] = float(percentage)
    return demographics_dict

In [16]:
def clean_turnout(turnout_string):
    '''
    Parse turnout number and percentage
    '''
    
    turnout_regex = re.compile(r"(?P<turnout>[\d,]+)\s\((?P<percentage>[\d.]+)\%\)")
    groups_dict = turnout_regex.search(turnout_string).groupdict()
    turnout = groups_dict['turnout']
    percentage = groups_dict['percentage']
    turnout = parse_comma_sep_num(turnout)
    percentage = float(percentage)
    return {
        "voter_turnout": turnout,
        "voter_turnout_percentage": percentage
    }

In [17]:
def clean_facts(raw_df):
    '''
    Clean info about each constituency
    '''
    
    clean_df = pd.DataFrame(columns=['registered_voters', 'majority', 'spoilt_votes',
       'unreturned_votes', 'voter_turnout', 'voter_turnout_percentage', 'parliament_code_digits',
       'constituency', 'state'])
    
    for index, row in raw_df.iterrows():
        row_dict = {}
        
        row_dict['registered_voters'] = parse_comma_sep_num(row['registered_voters'])
        row_dict['majority'] = parse_comma_sep_num(row['majority'])
        row_dict['spoilt_votes'] = parse_comma_sep_num(row['spoilt_votes'])
        row_dict['unreturned_votes'] = parse_comma_sep_num(row['unreturned_votes'])
        
        turnout_dict = clean_turnout(row['voter_turnout'])
        row_dict.update(turnout_dict)
        
        demographics_dict = clean_demographics(row['demographics'])
        row_dict.update(demographics_dict)
        
        row_dict['parliament_code_digits'] = clean_consituency_code(row)
        
        row_dict['constituency'] = row['panel_name']
        
        ## Split upper camel case string into individual words
        row_dict['state'] = ''.join(' ' + char if char.isupper() else char.strip() for char in row['state_region']).strip()
        
                
        ## Clean Typos: 
        ### "state" column: "Negri Sembilan" -> "Negeri Sembilan"
        
        row_dict = fix_typos([
            { 'key': 'state', 'old_value': 'Negri Sembilan', 'new_value': 'Negeri Sembilan'}
        ],row_dict)
        
        
        clean_df = clean_df.append(row_dict, ignore_index=True)
        clean_df.fillna(0, inplace=True)
        
    
    return clean_df

In [18]:
state_facts = clean_facts(append_all(state_fact_paths))

In [19]:
parliament_facts = clean_facts(append_all(parliament_fact_paths))

In [20]:
state_facts.to_csv('data/cleaned/the_star/constituency_info_'+str(election_year)+'_states.csv')

In [21]:
parliament_facts.to_csv('data/cleaned/the_star/constituency_info_'+str(election_year)+'_parliament.csv')

## Code Book

### Prepare Code Book for results data

In [22]:
state_votes.head(5)

,name,coalition,party_code,votes,vote_share,parliament_code_digits,constituency,state,vote_share_of_registered_voters,winner,total_votes
0,Mohammad Mohamarin,,WARISAN,3613,47.791005,167-001,BANGGI (P167),Sabah,32.7,1,7560
1,Abd Mijul Unaini,BN,UMNO,3234,42.777778,167-001,BANGGI (P167),Sabah,29.3,0,7560
2,Norlaji Amir Hassan,,Sabah STAR,367,4.854497,167-001,BANGGI (P167),Sabah,3.3,0,7560
3,Abidula Amsana,,PHRS,198,2.619048,167-001,BANGGI (P167),Sabah,1.8,0,7560
4,Kusugan Ali,,PPRS,105,1.388889,167-001,BANGGI (P167),Sabah,0.9,0,7560


In [23]:
parliament_votes.head(5)

,name,coalition,party_code,votes,vote_share,parliament_code_digits,constituency,state,vote_share_of_registered_voters,winner,total_votes
0,Mordi Bimol,PH,DAP,12771,56.714628,192,MAS GADING,Sarawak,43.1,1,22518
1,Nogeh Gumbek,BN,PDP,9747,43.285372,192,MAS GADING,Sarawak,32.9,0,22518
2,Wan Junaidi Tuanku Jaafar,BN,PBB,26379,79.280498,193,SANTUBONG,Sarawak,57.8,1,33273
3,Mohamad Fidzuan Zaidi,PH,AMANAH,6894,20.719502,193,SANTUBONG,Sarawak,15.1,0,33273
4,Fadillah Yusof,BN,PBB,28306,65.912213,194,PETRA JAYA,Sarawak,48.9,1,42945


In [24]:
state_votes.columns

Index(['name', 'coalition', 'party_code', 'votes', 'vote_share',
       'parliament_code_digits', 'constituency', 'state',
       'vote_share_of_registered_voters', 'winner', 'total_votes'],
      dtype='object')

In [25]:
len(state_votes.columns)

11

In [26]:
results_codebook_dict = [
    {
        "Variable Name": "name",
        "Variable Description": "Name of candidate",
        "Variable Type": "string"
    },
    {
        "Variable Name": "coalition",
        "Variable Description": "Short name of the coalition that the candidate's party belongs to",
        "Variable Type": "string"
    },
    {
        "Variable Name": "party_code",
        "Variable Description": "Short name of the coalition that the candidate's party belongs to",
        "Variable Type": "string"
    },
    {
        "Variable Name": "votes",
        "Variable Description": "Number of votes awarded to the candidate",
        "Variable Type": "number"
    },
    {
        "Variable Name": "vote_share",
        "Variable Description": "Percentage of votes awarded to the candidate in this constituency",
        "Variable Type": "number (percentage)"
    },
    {
        "Variable Name": "parliament_code_digits",
        "Variable Description": "For parliamentary constituencies: 3 digit code of constituency " +
            "(e.g. P125 for Putrajaya is coded 125). " +
            "For state constituencies: 3 digit parliament code followed by 3 digit state constituency code " +
            "(e.g. N33 for Air Itam (P51) in Penang is coded 051-033)",
        "Variable Type": "string (3 numerical digits for parliament, 3-3 for state)"
    },
    {
        "Variable Name": "constituency",
        "Variable Description": "Name of the constituency",
        "Variable Type": "string"
    },
    {
        "Variable Name": "state",
        "Variable Description": "Name of the state that the constituency belongs to",
        "Variable Type": "string"
    },
    {
        "Variable Name": "vote_share_of_registered_voters",
        "Variable Description": "Vote share of candidate as a percentage of all registered voters. "
            + "This is included because the original vote share data in The Star's website was calculated in this way",
        "Variable Type": "number (percentage)"
    },
    {
        "Variable Name": "winner",
        "Variable Description": "Boolean indicating whether a candidate won in the constituency. " +
            "Coded 1 if they are a winner, 0 if they are not.",
        "Variable Type": "number (1 or 0)"
    },
    {
        "Variable Name": "total_votes",
        "Variable Description": "Sum of the votes that were awarded to all the candidates in that constituency.",
        "Variable Type": "number"
    },
]


In [27]:
results_codebook_df = pd.DataFrame(results_codebook_dict)

In [28]:
results_codebook_df.to_csv('data/cleaned/the_star/results_'+str(election_year)+'_CODEBOOK.csv')

### Prepare Code Book for constituency information data

In [29]:
state_facts.head(5)

,registered_voters,majority,spoilt_votes,unreturned_votes,voter_turnout,voter_turnout_percentage,parliament_code_digits,constituency,state,ethnicity_B/P Sabah,ethnicity_B/P Sarawak,ethnicity_Chinese,ethnicity_Indian,ethnicity_Malay,ethnicity_Orang Asli (Pen. Msia),ethnicity_Others,ethnicity_Muslim bumiputra
0,10111,142,182,0,8018,79.3,001-001,TITI TINGGI (P1),Perlis,0.08,0.14,19.29,2.13,77.83,0.01,0.53,0.0
1,8914,416,128,0,7027,78.8,001-002,BESERI (P1),Perlis,0.08,0.21,16.83,0.74,81.33,0.00,0.81,0.0
2,10542,1367,221,0,8624,81.8,001-003,CHUPING (P1),Perlis,0.01,0.02,1.98,0.58,85.78,0.00,11.63,0.0
3,7617,720,146,0,6339,83.2,001-004,MATA AYER (P1),Perlis,1.58,3.61,2.59,0.70,90.77,0.08,0.68,0.0
4,8912,949,102,0,0,0.0,001-005,SANTAN (P1),Perlis,0.03,0.02,0.44,0.10,99.28,0.00,0.12,0.0


In [30]:
parliament_facts.head(5)

,registered_voters,majority,spoilt_votes,unreturned_votes,voter_turnout,voter_turnout_percentage,parliament_code_digits,constituency,state,ethnicity_B/P Sabah,ethnicity_B/P Sarawak,ethnicity_Chinese,ethnicity_Indian,ethnicity_Malay,ethnicity_Orang Asli (Pen. Msia),ethnicity_Others,ethnicity_Muslim bumiputra
0,48522,2045,586,0.0,39733,81.9,126,JELEBU,Negeri Sembilan,0.25,0.15,24.13,5.69,65.01,4.58,0.19,0.0
1,72122,1631,1025,0.0,58523,81.1,127,JEMPOL,Negeri Sembilan,0.06,0.05,23.24,12.31,63.35,0.77,0.21,0.0
2,110168,30694,1052,0.0,93254,84.6,128,SEREMBAN,Negeri Sembilan,0.19,0.13,36.75,13.25,48.39,0.70,0.58,0.0
3,49801,200,769,0.0,0,0.0,129,KUALA PILAH,Negeri Sembilan,0.07,0.10,15.84,4.86,77.01,1.89,0.23,0.0
4,102838,46867,303,0.0,86655,84.3,130,RASAH,Negeri Sembilan,0.18,0.21,47.79,20.95,29.65,0.41,0.80,0.0


In [31]:
state_facts.columns

Index(['registered_voters', 'majority', 'spoilt_votes', 'unreturned_votes',
       'voter_turnout', 'voter_turnout_percentage', 'parliament_code_digits',
       'constituency', 'state', 'ethnicity_B/P Sabah', 'ethnicity_B/P Sarawak',
       'ethnicity_Chinese', 'ethnicity_Indian', 'ethnicity_Malay',
       'ethnicity_Orang Asli (Pen. Msia)', 'ethnicity_Others',
       'ethnicity_Muslim bumiputra'],
      dtype='object')

In [32]:
len(state_facts.columns)

17

In [33]:
constituency_info_codebook_dict = [
    {
        "Variable Name": "registered_voters",
        "Variable Description": "Number of registered voters in the constituency",
        "Variable Type": "number"
    },
    {
        "Variable Name": "majority",
        "Variable Description": "The majority of votes by which the winning candidate won",
        "Variable Type": "number"
    },
    {
        "Variable Name": "spoilt_votes",
        "Variable Description": "The number of sploit votes",
        "Variable Type": "number"
    },
    {
        "Variable Name": "unreturned_votes",
        "Variable Description": "The number of unreturned votes",
        "Variable Type": "number"
    },
    {
        "Variable Name": "voter_turnout",
        "Variable Description": "Voter turnout in number of votes",
        "Variable Type": "number"
    },
    {
        "Variable Name": "voter_turnout_percentage",
        "Variable Description": "Voter turnout as a percentage of registered voters",
        "Variable Type": "number (percentage)"
    },
    {
        "Variable Name": "parliament_code_digits",
        "Variable Description": "For parliamentary constituencies: 3 digit code of constituency " +
            "(e.g. P125 for Putrajaya is coded 125). " +
            "For state constituencies: 3 digit parliament code followed by 3 digit state constituency code " +
            "(e.g. N33 for Air Itam (P51) in Penang is coded 051-033)",
        "Variable Type": "string (3 numerical digits for parliament, 3-3 for state)"
    },
    {
        "Variable Name": "constituency",
        "Variable Description": "Name of the constituency",
        "Variable Type": "string"
    },
    {
        "Variable Name": "state",
        "Variable Description": "Name of the state that the constituency belongs to",
        "Variable Type": "string"
    },
    {
        "Variable Name": "ethnicity_*",
        "Variable Description": "All variables that start with 'ethnicity_' shows the percentage of " +
            "registered voters who belong to each ethnicity",
        "Variable Type": "number (percentage)"
    }
]

In [34]:
constituency_info_codebook_df = pd.DataFrame(constituency_info_codebook_dict)

In [35]:
constituency_info_codebook_df

,Variable Name,Variable Description,Variable Type
0,registered_voters,Number of registered voters in the constituency,number
1,majority,The majority of votes by which the winning can...,number
2,spoilt_votes,The number of sploit votes,number
3,unreturned_votes,The number of unreturned votes,number
4,voter_turnout,Voter turnout in number of votes,number
5,voter_turnout_percentage,Voter turnout as a percentage of registered vo...,number (percentage)
6,parliament_code_digits,For parliamentary constituencies: 3 digit code...,"string (3 numerical digits for parliament, 3-3..."
7,constituency,Name of the constituency,string
8,state,Name of the state that the constituency belong...,string
9,ethnicity_*,All variables that start with 'ethnicity_' sho...,number (percentage)


In [36]:
constituency_info_codebook_df.to_csv('data/cleaned/the_star/constituency_info_'+str(election_year)+'_CODEBOOK.csv')